# Running remotely in Vulkan Engine

Now that we've tested and run our policy locally, we can use Vulkan Engine to scale and serve it. \
The same code will run in the remote server, and it'll have the added benefits of scalability, monitoring and governance. 

This notebook will show you how to upload this policy to the server and run a backtest!

In [ ]:
import os

import pandas as pd

from vulkan_public.cli import client as vulkan
from vulkan_public.cli.context import Context

In [ ]:
ctx = Context()

## Creating the Data Source

As we're using a file to get the scores, we need to upload it. \
Then we'll let the platform know where to find the data by creating a remote data source with it.

In [ ]:
file_info = vulkan.backtest.upload_backtest_file(
    ctx,
    file_name="Data Source Reference",
    file_path="file_data_source.parquet",
    file_format="PARQUET",
    schema={"tax_id": "str", "score": "int"},
)
file_id = file_info["uploaded_file_id"]

In [ ]:
remote_file_source_schema = {
    # The name of our Data Source. This is the same name we used when creating the Policy.
    "name": "file-input:my-api-name:v0.0.1",
    # Keys are used to know how to "look up" information. In this case, we know we should send the Tax Id column.
    "keys": ["tax_id"],
    # Where the magic happens: here, we're telling Vulkan to use our file as the information source.
    "source": {"file_id": file_id},
}

file_data_source_id = vulkan.data.create_data_source(
    ctx,
    config=remote_file_source_schema,
)

## Creating the Policy

In [ ]:
policy_path = os.path.abspath("..")

policy_id = vulkan.policy.create_policy(ctx, "Test Beam Policy")
policy_version_id = vulkan.policy.create_policy_version(
    ctx,
    policy_id=policy_id,
    version_name="My First Version",
    repository_path=policy_path,
)

In [ ]:
vulkan.policy_version.create_backtest_workspace(ctx, policy_version_id)

## Running a Backtest

Now we can create full-scale backtests using Vulkan. \
This is very similar to doing a batch run, except we use scalable infrastructure and keep track of performance for you. \
You can also specify some additional information on your "target" variable to get automatically generated metrics.

In [ ]:
backtest_info = vulkan.backtest.create_backtest(
    ctx,
    policy_version_id=policy_version_id,
    input_file_id=file_id,
    config_variables=[
        {"SCORE_CUTOFF": 500},
        {"SCORE_CUTOFF": 700},
    ],
    metrics_config={
        "target_column": "default",
    },
)

backtest_id = backtest_info["backtest_id"]

In [ ]:
vulkan.backtest.poll_backtest_status(ctx, backtest_id, timeout=360)

In [ ]:
output = vulkan.backtest.get_results(ctx, backtest_id)
output_data = pd.DataFrame(output)
output_data.head(10)